Integración con Google Cloud Functions

    Cree una función en Google Cloud Functions:
        Ingrese a la consola de Google Cloud.
        Vaya a la sección "Cloud Functions".
        Haga clic en "Crear función".

    Configure la función:
        Asigne un nombre a la función.
        Seleccione un disparador HTTP o un evento de Cloud Storage (por ejemplo, creación de objeto en Google Cloud Storage).
        Establezca el tiempo de ejecución y otros parámetros.

    Suba el script:
        En la sección "Código fuente", elija "Editor en línea" o suba un archivo comprimido con el script.
        Pegue el script en el archivo main.py.
        Cree un archivo requirements.txt y agregue las dependencias:



In [ ]:
import pandas as pd
import os
from google.cloud import storage

def clean_and_process_files(bucket_name, source_folder, destination_folder):
    # Inicialización del cliente de Google Cloud Storage
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)

    # Función para leer y procesar un archivo CSV
    def process_file(file_path):
        # Carga de datos
        df = pd.read_csv(file_path)

        # Limpieza de datos eliminando filas con valores nulos
        df.dropna(inplace=True)

        # Estandarización de los nombres de las columnas
        df.columns = df.columns.str.lower().str.replace(' ', '_')

        # Devolución del DataFrame procesado
        return df

    # Función para el procesamiento específico de order_items
    def process_order_items(df):
        # Conversión de fechas
        df['shipping_limit_date'] = pd.to_datetime(df['shipping_limit_date'])
        # Creación de nuevas columnas de fecha y hora
        df['shipping_date'] = df['shipping_limit_date'].dt.date
        df['shipping_time'] = df['shipping_limit_date'].dt.time
        return df

    # Función para el procesamiento específico de reviews
    def process_reviews(df):
        # Eliminación de filas con valor 's' en la columna score
        df = df[df['score'] != 's']
        # Conversión de la columna score a tipo entero
        df['score'] = df['score'].astype(int)
        return df

    # Función para el procesamiento específico de orders
    def process_orders(df):
        # Rellenar valores nulos en las columnas específicas con una fecha por defecto
        df['approved_at'].fillna('1970-01-01 00:00:00', inplace=True)
        df['delivered_courier_date'].fillna('1970-01-01 00:00:00', inplace=True)
        df['delivered_customer_date'].fillna('1970-01-01 00:00:00', inplace=True)
        return df

    # Lista de archivos para procesar
    files = [
        'order_items.csv', 'products.csv', 'reviews.csv', 'sellers.csv',
        'customers.csv', 'geolocalizaciones.csv', 'orders.csv', 'order_payments.csv'
    ]

    # Procesamiento de cada archivo
    for file_name in files:
        # Descarga del archivo desde Google Cloud Storage
        blob = bucket.blob(os.path.join(source_folder, file_name))
        blob.download_to_filename(f'/tmp/{file_name}')

        # Lectura y procesamiento del archivo
        df = process_file(f'/tmp/{file_name}')

        # Procesamiento específico para ciertos archivos
        if file_name == 'order_items.csv':
            df = process_order_items(df)
        elif file_name == 'reviews.csv':
            df = process_reviews(df)
        elif file_name == 'orders.csv':
            df = process_orders(df)

        # Guardado del archivo procesado de vuelta en Google Cloud Storage
        df.to_csv(f'/tmp/processed_{file_name}', index=False)
        processed_blob = bucket.blob(os.path.join(destination_folder, f'processed_{file_name}'))
        processed_blob.upload_from_filename(f'/tmp/processed_{file_name}')


